In [72]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords,words
import numpy as np
import seaborn as sns
import re

#### We are setting the parameter index_col=0 to drop unnamed column

In [73]:
data=pd.read_excel(r'orginal_data_from_glassdoor2.xlsx')
data.head()

,City,JobId,Source,CollectedDate,JobTitle,CompanyName,RatingNumber,PostedDate,Posted_Date_N,Salary,jobURL,fullJobDescribtion,Size,Type_Of_Ownership,Sector,Founded,Industry,Revenue
0,Remote,1008266954603,Glassdoor,2022-11-09,Data Scientist,Creatura Consulting\n5.0,5.0,24h,2022-10-16,$45.00 - $75.00 Per Hour(Employer est.),https://www.glassdoor.com/partner/jobListing.h...,The ideal candidate's favorite words are learn...,1 to 50 Employees,Company - Private,NaN,NaN,NaN,Unknown / Non-Applicable
1,"Alpharetta, GA",1008266055464,Glassdoor,2022-11-09,Jr. Data Scientist (Remote),RELX\n3.9,3.9,24h,2022-10-16,$54K - $77K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,This position can be home-based anywhere in th...,10000+ Employees,Company - Public,Information Technology,1880.0,Information Technology Support Services,$10+ billion (USD)
2,Remote,1008259480518,Glassdoor,2022-11-09,Data Scientist- Coding Dojo Hiring Partner,CODING DOJO INC\n3.9,3.9,4d,2022-11-05,NaN,https://www.glassdoor.com/partner/jobListing.h...,Title: Data Scientist\nLocation: Remote\nType:...,51 to 200 Employees,Company - Private,Education,2013.0,Education & Training Services,$5 to $25 million (USD)
3,"Moline, IL",1008266830132,Glassdoor,2022-11-09,Data Scientist - REMOTE,John Deere\n4.1,4.1,24h,2022-10-16,$86K - $129K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,There are over 7 billion people on this planet...,10000+ Employees,Company - Public,Manufacturing,1837.0,Machinery Manufacturing,$10+ billion (USD)
4,"Washington, DC",1008253713007,Glassdoor,2022-11-09,Junior Data Scientist,World Bank Group\n4.2,NaN,6d,2022-11-03,$70K - $108K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,Job #:\nreq19774\n\nOrganization:\nWorld Bank\...,10000+ Employees,Nonprofit Organization,NaN,NaN,NaN,$1 to $5 billion (USD)


In [74]:
data.columns

Index(['City', 'JobId', 'Source', 'CollectedDate', 'JobTitle', 'CompanyName',
       'RatingNumber', 'PostedDate', 'Posted_Date_N', 'Salary', 'jobURL',
       'fullJobDescribtion', 'Size', 'Type_Of_Ownership', 'Sector', 'Founded',
       'Industry', 'Revenue'],
      dtype='object')

In [75]:
data.shape

(2268, 18)

In [76]:
data

,City,JobId,Source,CollectedDate,JobTitle,CompanyName,RatingNumber,PostedDate,Posted_Date_N,Salary,jobURL,fullJobDescribtion,Size,Type_Of_Ownership,Sector,Founded,Industry,Revenue
0,Remote,1008266954603,Glassdoor,2022-11-09,Data Scientist,Creatura Consulting\n5.0,5.0,24h,2022-10-16,$45.00 - $75.00 Per Hour(Employer est.),https://www.glassdoor.com/partner/jobListing.h...,The ideal candidate's favorite words are learn...,1 to 50 Employees,Company - Private,NaN,NaN,NaN,Unknown / Non-Applicable
1,"Alpharetta, GA",1008266055464,Glassdoor,2022-11-09,Jr. Data Scientist (Remote),RELX\n3.9,3.9,24h,2022-10-16,$54K - $77K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,This position can be home-based anywhere in th...,10000+ Employees,Company - Public,Information Technology,1880.0,Information Technology Support Services,$10+ billion (USD)
2,Remote,1008259480518,Glassdoor,2022-11-09,Data Scientist- Coding Dojo Hiring Partner,CODING DOJO INC\n3.9,3.9,4d,2022-11-05,NaN,https://www.glassdoor.com/partner/jobListing.h...,Title: Data Scientist\nLocation: Remote\nType:...,51 to 200 Employees,Company - Private,Education,2013.0,Education & Training Services,$5 to $25 million (USD)
3,"Moline, IL",1008266830132,Glassdoor,2022-11-09,Data Scientist - REMOTE,John Deere\n4.1,4.1,24h,2022-10-16,$86K - $129K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,There are over 7 billion people on this planet...,10000+ Employees,Company - Public,Manufacturing,1837.0,Machinery Manufacturing,$10+ billion (USD)
4,"Washington, DC",1008253713007,Glassdoor,2022-11-09,Junior Data Scientist,World Bank Group\n4.2,NaN,6d,2022-11-03,$70K - $108K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,Job #:\nreq19774\n\nOrganization:\nWorld Bank\...,10000+ Employees,Nonprofit Organization,NaN,NaN,NaN,$1 to $5 billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,Remote,1007751227553,Glassdoor,2022-11-14,Junior Data Scientist-Remote,Sabre Systems Inc.\n4.7,4.7,30d+,2022-10-15,NaN,https://www.glassdoor.com/partner/jobListing.h...,Responsibilities:\nThe NAVAIR Digital Group le...,201 to 500 Employees,Company - Private,Aerospace & Defense,1989.0,Aerospace & Defense,$25 to $100 million (USD)
2264,"Lake Forest, IL",1008263900986,Glassdoor,2022-11-14,Associate Data Scientist,Abbott Laboratories\n3.8,3.8,6d,2022-11-08,$83K - $120K (Glassdoor est.),https://www.glassdoor.com/partner/jobListing.h...,Abbott is a global healthcare leader that help...,10000+ Employees,Company - Public,Healthcare,1888.0,Health Care Services & Hospitals,$10+ billion (USD)
2265,Remote,1008238407975,Glassdoor,2022-11-14,Data Scientist (100% Remote) PERM AMICA,Access to future,NaN,17d,2022-10-28,$110K - $115K (Employer est.),https://www.glassdoor.com/partner/jobListing.h...,Number of openings: 1\nSalary: $115k PERM (som...,Unknown,Company - Public,NaN,NaN,NaN,Unknown / Non-Applicable
2266,"New York, NY",1008102598975,Glassdoor,2022-11-14,Data Scientist,CVS Health\n3.1,3.1,30d+,2022-10-15,$60K - $129K (Employer est.),https://www.glassdoor.com/partner/jobListing.h...,Description/Fundamental Components:\n\nDevelop...,10000+ Employees,Company - Public,Healthcare,1963.0,Health Care Services & Hospitals,$10+ billion (USD)


In [77]:
def missing_values_table(df):
    mis_val=df.isnull().sum()
    mis_val_percent = mis_val *100 / len(df)
    mis_val_table= pd.concat([mis_val,mis_val_percent],axis=1)
    mis_val_table_ren_columns= mis_val_table.rename(columns={0:'Missing Values',1:'% Total Values'})
    #sort by column
    mis_val_table_ren_columns =mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% Total Values',ascending=False).round(1)
    print("Your selected dataset has "+str(df.shape[1])+" columns and "+str(len(df))+" observation .\n"
          "There are "+str(mis_val_table_ren_columns.shape[0])+" columns that have missing value")
    return mis_val_table_ren_columns


In [78]:
missing_values_table(data)

Your selected dataset has 18 columns and 2268 observation .
There are 14 columns that have missing value


,Missing Values,% Total Values
Founded,992,43.7
Sector,782,34.5
Industry,782,34.5
RatingNumber,618,27.2
Salary,460,20.3
Size,391,17.2
Type_Of_Ownership,391,17.2
Revenue,391,17.2
fullJobDescribtion,112,4.9
City,78,3.4


In [79]:
data=data.dropna(axis=0)
data.shape

(955, 18)

In [80]:
data['Salary'].head()

1       $54K - $77K (Glassdoor est.)
3      $86K - $129K (Glassdoor est.)
5      $99K - $144K (Glassdoor est.)
7     $114K - $164K (Glassdoor est.)
10      $60K - $129K (Employer est.)
Name: Salary, dtype: object

#### Feature engineering on the salary column

In [81]:
salary=data['Salary'].apply(lambda x:x.split('(')[0])

In [82]:
data['hourly'] = data['Salary'].apply(lambda x:1 if 'per hour' in x.lower() else 0)

C:\Users\Mohameed\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
data['hourly'].value_counts()

0    747
1    208
Name: hourly, dtype: int64

#### as we can see above there are the dollar sign and thousand marker ,So we are removing'

In [84]:
minus_kd=salary.apply(lambda x:x.replace('$','').replace('K',''))

In [85]:
minus_kd.head(30)

1       54 - 77 
3      86 - 129 
5      99 - 144 
7     114 - 164 
10     60 - 129 
16     67 - 120 
17     84 - 129 
18     91 - 122 
21     82 - 117 
23    112 - 218 
25    105 - 170 
26     83 - 121 
27     84 - 123 
28     97 - 138 
29    125 - 195 
30    150 - 160 
32     96 - 143 
34    114 - 172 
35     96 - 220 
36     85 - 121 
37     80 - 112 
39    111 - 159 
40     92 - 142 
44    102 - 154 
45    103 - 154 
47     79 - 125 
48    144 - 176 
50     80 - 122 
51     76 - 108 
52    100 - 120 
Name: Salary, dtype: object

#### Our next step let's make a column for a min for salary in each post if we look at min_hr we can see that they're separated by '-',so we are going to split on the '-' and just take the first thing

In [86]:
data['min_salary']=minus_kd.apply(lambda x:x.split('-')[0])
data[['Salary','min_salary']].head()

C:\Users\Mohameed\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Salary,min_salary
1,$54K - $77K (Glassdoor est.),54
3,$86K - $129K (Glassdoor est.),86
5,$99K - $144K (Glassdoor est.),99
7,$114K - $164K (Glassdoor est.),114
10,$60K - $129K (Employer est.),60


#### Our next step let's make a column for a max for salary in each post if we look at min_hr we can see that they're separated by '-',so we are going to split on the '-' and just take the first thing

In [87]:
data['max_salary']=minus_kd.apply(lambda x:x.split('-')[1])
data[['Salary','max_salary']].head()

IndexError: list index out of range

#### let's check salary if there is a max is salary by create a function test take a column of salary as a prameter and return True if the salary have max and False otherwise using try and except

In [88]:
def test(val):
    try:
        val.split('-')[1]
    except:
        return False
    return True

#### We apply a function on salary feature ,but we usw tile ~ sympol which return all values dont have max_salary

In [89]:
data_without_max=data[~data['Salary'].apply(test)]
data_without_max['Salary']

484              $102K (Employer est.)
500     $60.00 Per Hour(Employer est.)
538     $70.00 Per Hour(Employer est.)
557     $40.00 Per Hour(Employer est.)
823     $17.85 Per Hour(Employer est.)
                     ...              
2124              $32K (Employer est.)
2127    $16.00 Per Hour(Employer est.)
2147    $45.00 Per Hour(Employer est.)
2154              $32K (Employer est.)
2157    $16.00 Per Hour(Employer est.)
Name: Salary, Length: 92, dtype: object

#### as we can see above there are 92 value not have max salary so we drop it from DataFrame

In [90]:
data=data.drop(data_without_max.index,axis=0)

In [91]:
data[~data['Salary'].apply(test)]


,City,JobId,Source,CollectedDate,JobTitle,CompanyName,RatingNumber,PostedDate,Posted_Date_N,Salary,jobURL,fullJobDescribtion,Size,Type_Of_Ownership,Sector,Founded,Industry,Revenue,hourly,min_salary


In [92]:
salary=data['Salary'].apply(lambda x:x.split('(')[0])
minus_kd=salary.apply(lambda x:x.replace('$','').replace('K',''))
min_hr=minus_kd.apply(lambda x:x.lower().replace('per hour','').replace(' ',''))

In [93]:
data['max_salary']=min_hr.apply(lambda x:x.split('-')[1])
data[['Salary','max_salary']].head()

,Salary,max_salary
1,$54K - $77K (Glassdoor est.),77
3,$86K - $129K (Glassdoor est.),129
5,$99K - $144K (Glassdoor est.),144
7,$114K - $164K (Glassdoor est.),164
10,$60K - $129K (Employer est.),129


#### we want to create a new feature called avg_salary based on min_salary and max_salary which calculating the average for salary ,but firstly must check the data type of min salary and max salary is int ?

In [94]:
print('the data type of min salary column  :', data.min_salary.dtype)
print('the data type of max salary column  :', data.max_salary.dtype)

the data type of min salary column  : object
the data type of max salary column  : object


#### as we can see above the data type of min salary and max salary columns are Object which there are some values are string so we can drop it So we write a function is_int() with min_salary as an argument and return all the integering

In [95]:
def is_int(df):
    try:
        int(df)
    except:
        return False
    return True

In [96]:
data[~data['min_salary'].apply(is_int)]['min_salary']

73      60.00 
173     58.00 
208     60.00 
235     60.00 
420     85.00 
         ...  
2149    30.00 
2155    30.00 
2158    17.00 
2221    32.00 
2228    65.00 
Name: min_salary, Length: 131, dtype: object

#### firstly we will convert to float after that convert to int to avoid ValueError: invalid literal for int() with base 10:

In [97]:
data['min_salary']= data['min_salary'].astype(float).astype(int)
data['max_salary']= data['max_salary'].astype(float).astype(int)

#### Check if all values of min_salary and max_salary features are integers ,as we can see below the data type of min_salary is int and the data type of max_salary is int ,So all values of min_salary and max_salary are integers

In [98]:
print(data['min_salary'].dtypes)
print(data['max_salary'].dtypes)

int32
int32


In [99]:
data['min_salary']=data['min_salary'].astype(int)
data['max_salary']=data['max_salary'].astype(int)
data['avg_salary']=(data['max_salary']+data['min_salary'])/2
data.head()

,City,JobId,Source,CollectedDate,JobTitle,CompanyName,RatingNumber,PostedDate,Posted_Date_N,Salary,...,Size,Type_Of_Ownership,Sector,Founded,Industry,Revenue,hourly,min_salary,max_salary,avg_salary
1,"Alpharetta, GA",1008266055464,Glassdoor,2022-11-09,Jr. Data Scientist (Remote),RELX\n3.9,3.9,24h,2022-10-16,$54K - $77K (Glassdoor est.),...,10000+ Employees,Company - Public,Information Technology,1880.0,Information Technology Support Services,$10+ billion (USD),0,54,77,65.5
3,"Moline, IL",1008266830132,Glassdoor,2022-11-09,Data Scientist - REMOTE,John Deere\n4.1,4.1,24h,2022-10-16,$86K - $129K (Glassdoor est.),...,10000+ Employees,Company - Public,Manufacturing,1837.0,Machinery Manufacturing,$10+ billion (USD),0,86,129,107.5
5,"Mountain View, CA",1008124177255,Glassdoor,2022-11-09,"Associate Data Scientist, 2023 Graduate",Atlassian\n4.7,4.7,30d+,2022-10-10,$99K - $144K (Glassdoor est.),...,5001 to 10000 Employees,Company - Public,Information Technology,2002.0,Software Development,Unknown / Non-Applicable,0,99,144,121.5
7,"Redmond, WA",1008266687928,Glassdoor,2022-11-09,Data Scientist,Microsoft\n4.4,4.4,24h,2022-10-16,$114K - $164K (Glassdoor est.),...,10000+ Employees,Company - Public,Information Technology,1975.0,Computer Hardware Development,$10+ billion (USD),0,114,164,139.0
10,Illinois,1008266764955,Glassdoor,2022-11-09,Data Scientist,CVS Health\n3.1,3.1,24h,2022-10-16,$60K - $129K (Employer est.),...,10000+ Employees,Company - Public,Healthcare,1963.0,Health Care Services & Hospitals,$10+ billion (USD),0,60,129,94.5


#### Now we can drop min_salary,max_salary in favor of avg_salary

In [100]:
data=data.drop(['min_salary','max_salary'],axis=1)

#### Now we want to dispaly CompanyName

In [101]:
data['CompanyName'].head()

1           RELX\n3.9
3     John Deere\n4.1
5      Atlassian\n4.7
7      Microsoft\n4.4
10    CVS Health\n3.1
Name: CompanyName, dtype: object

#### Now we want to create a new feature company_text based on company Name which contain a name of company text only without numbers

In [102]:
data['company_text']=data['CompanyName'].apply(lambda x:x[:-3])

In [103]:
data['company_text'].head()

1           RELX\n
3     John Deere\n
5      Atlassian\n
7      Microsoft\n
10    CVS Health\n
Name: company_text, dtype: object

#### as we can see above there is a \n at the end of the company text ,so we must to replace it with a space

In [104]:
for col in data[['company_text','fullJobDescribtion']].columns:
    data[col] = data[col].apply(lambda x:x.replace('\n',' '))
data['company_text'].head()

1           RELX 
3     John Deere 
5      Atlassian 
7      Microsoft 
10    CVS Health 
Name: company_text, dtype: object

#### What is the state of job ?
We have the location in the following format 'Houston, TX' we will just take location and we are going do a split and take the second part

In [105]:
def find_state(val):
    try:
        return val.split(',')[1]
    except:
        return val

In [106]:
data['job_state'] = data['City'].apply(find_state)
data[['City','job_state']].head()


,City,job_state
1,"Alpharetta, GA",GA
3,"Moline, IL",IL
5,"Mountain View, CA",CA
7,"Redmond, WA",WA
10,Illinois,Illinois


#### What is the city of a job ?
We have the location in the follwing format 'Redmond, WA' we will just take the location and we're gonna do a split and take the first part 

In [107]:
data['job_city'] = data['City'].apply(lambda x: x.split(',')[0])

#### let's check the columns of job_city 

In [108]:
data[['City','job_city']].head()

,City,job_city
1,"Alpharetta, GA",Alpharetta
3,"Moline, IL",Moline
5,"Mountain View, CA",Mountain View
7,"Redmond, WA",Redmond
10,Illinois,Illinois


#### Now we can drop Location in favor of job_state,job_city

In [109]:
data=data.drop('City',axis=1)

In [110]:
data['job_state'].value_counts()

 CA               188
Remote            167
 NY                73
 IL                50
 GA                40
 TX                40
 MA                31
 WA                30
 MD                24
 VA                20
 NJ                20
 CT                19
 NC                19
 MO                15
 PA                12
 AL                12
 TN                11
 OR                10
North Carolina      9
Virginia            9
 FL                 8
 DE                 7
Illinois            7
 DC                 5
 MI                 5
 IA                 4
 OH                 4
 UT                 4
 MS                 3
 CO                 3
 NV                 3
 AR                 3
 AZ                 2
 NM                 2
Manhattan           2
 SC                 1
 ME                 1
Name: job_state, dtype: int64

In [111]:
data.columns

Index(['JobId', 'Source', 'CollectedDate', 'JobTitle', 'CompanyName',
       'RatingNumber', 'PostedDate', 'Posted_Date_N', 'Salary', 'jobURL',
       'fullJobDescribtion', 'Size', 'Type_Of_Ownership', 'Sector', 'Founded',
       'Industry', 'Revenue', 'hourly', 'avg_salary', 'company_text',
       'job_state', 'job_city'],
      dtype='object')

#### We want to create a new feature called 'age_company' it contains the age of company 

In [112]:
data['age_company'] = 2022-data['Founded'].astype('int')
data['age_company'].head()

1     142
3     185
5      20
7      47
10     59
Name: age_company, dtype: int32

#### We will use NLTK library to import stopwords and dispaly them

In [113]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words=stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mohameed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mohameed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### let's disply the length first of job description in our data 

In [114]:
len(data['fullJobDescribtion'][1])

3590

#### as we can see above there is dations@relx.com and 1.855.833.5120 this it's not useful in analysis  ,We want to create a function preprocessing with a text as a parameter using Regular Expression to remove charecters, decimal numbers ,which I keep alphanumeric characters and replace all other charecters with blank, as well as duplicated words and stop words these are a series of words that no add information to our mode

In [115]:
def preprocessing1(text):
    text=re.sub(r"[^a-zA-Z\s+]",' ',text.lower())
    tokens=[]
    duplicated_word=[]
    for token in nltk.wordpunct_tokenize(text):
        if (token in stop_words) or (token in duplicated_word) : 
            continue
        tokens.append(token)
        duplicated_word.append(token)
    return " ".join(tokens)

#### We apply preprocessing function on Job Title ,Job Description , Industry , Sector and Type of ownership

In [116]:
data_sub=data[['fullJobDescribtion','Industry','Sector','Type_Of_Ownership']]
for col in data_sub.columns:
    data[col] =data[col].apply(lambda x:preprocessing1(x))

#### We noticed the length of the first job description is after apply the function of preprocessing1 is 2256 this is pretty  

In [117]:
len(data['fullJobDescribtion'][1])

2256

In [118]:
def simplifier(title):
    if 'data scientist' in title.lower() or 'data science' in title.lower() or 'datascience' in title.lower() :
        return 'data scientist'
    elif 'machine learning engineer' in title.lower() or 'machinelearning' in title.lower()  or 'machine learning' in title.lower() or 'ml' in title.lower():
        return 'mle'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'data analyst' in title.lower():
        return 'data analyst'
    elif 'data entry' in title.lower():
        return 'data entry'
    elif 'maneger' in title.lower():
        return 'maneger'
    elif 'director' in title.lower():
        return 'director'
    elif 'Data Analytics' in title:
        return 'data analytics'
    elif 'business analyst' in title.lower():
        return 'business analysts'
    else:
        return 'other jobs related to data science'


In [119]:
def seniority(title):
    if 'senior' in title.lower() or 'sr' in title.lower() or 'sr.' in title.lower() or 'sr. ' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'entry level' in title.lower():
        return 'entry level'
    elif 'jenior' in title.lower() or 'junior' in title.lower() or 'jr' in title.lower() or 'jr.' in title.lower() or 'jr. ' in title.lower():
        return 'jenior'
    elif 'mid level' in title.lower() or 'med level' in title.lower():
        return 'mid level'
    return 'na'

#### Process data pased on above defined functions

In [120]:
data['job_simp']=data['JobTitle'].apply(simplifier)
data['seniority']=data['JobTitle'].apply(seniority)

In [121]:
data['job_simp'].value_counts()

data scientist                        310
data analyst                          164
data engineer                         129
data entry                            127
other jobs related to data science    117
data analytics                          7
business analysts                       4
mle                                     3
director                                2
Name: job_simp, dtype: int64

#### How many are job senior ,jenior and entry level in positions ?
as we can see below , we have 25 as a senior , 49 as a jenior ,12 as an entry level and we have a lot of like not specified .

In [122]:
data['seniority'].value_counts()

na             777
jenior          49
senior          25
entry level     12
Name: seniority, dtype: int64

#### we want to find a length job description for companies to know later in the Explority Data Analysis the companies that there is a positive correlation or not with salary  

In [123]:
data['desc_len'] = data['fullJobDescribtion'].apply(lambda x:len(x))

#### We want to create a new feature Machine Learning based on fullJobDescribtion to get total of job description which contain a Machine Learning

In [124]:
data['Machine_Learning'] = data['fullJobDescribtion'].apply(lambda x:1 if ('machine learning' in x.lower() or 'machinelearning' in x.lower()or 'ml' in x.lower()) else 0)

In [125]:
data['Machine_Learning'].value_counts()

0    521
1    342
Name: Machine_Learning, dtype: int64

#### We want to create a new feature Python based on Job Description to get total of job description which contain a Python

In [126]:
data['Python']=data['fullJobDescribtion'].apply(lambda x:1 if ('python' in x.lower() or 'py' in x.lower()) else 0 )

#### We want to create a new feature Spark based on Job Description to get total of job description which contain a Spark

In [127]:
data['Spark']=data['fullJobDescribtion'].apply(lambda x:1 if 'spark' in x.lower() else 0)

#### We want to create a new feature Hadoop based on Job Description to get total of job description which contain a Hadoop 

In [128]:
data['Hadoop']=data['fullJobDescribtion'].apply(lambda x:1 if 'hadoop' in x.lower() else 0)

#### We want to create a new feature Platform based on Job Description to get total of job description which contain a Plaform

In [129]:
data['Plarform'] =data['fullJobDescribtion'].apply(lambda x:1 if 'platform' in x.lower() else 0)

#### We want to create a new feature ETL based on Job Description to get total of job description which contain an ETL

In [130]:
data['ETL']=data['fullJobDescribtion'].apply(lambda x:1 if 'etl' in x.lower() else 0)

#### We want to create a new feature Sql based on Job Description to get total of job description which contain an Sql

In [131]:
data['Sql']=data['fullJobDescribtion'].apply(lambda x:1 if 'sql' in x.lower() else 0)

#### We want to create a new feature Secruity based on Job Description to get total of job description which contain a Security

In [132]:
data['Security']=data['fullJobDescribtion'].apply(lambda x:1 if 'security' in x.lower() else 0)

#### We want to create a new feature PHD based on Job Description to get total of job description which contain a PHD

In [133]:
data['PHD']=data['fullJobDescribtion'].apply(lambda x:1 if 'phd' in x.lower() else 0)

#### We want to create a new feature Excel based on Job Description to get total of job description which contain an Excel

In [134]:
data['Excel']=data['fullJobDescribtion'].apply(lambda x:1 if 'excel' in x.lower() else 0)

#### We want to create a new feature Aws based on Job Description to get total of job description which contain an Aws

In [135]:
data['Aws']=data['fullJobDescribtion'].apply(lambda x:1 if 'aws' in x.lower() else 0)

#### We want to create a new feature PyTorch based on Job Description to get total of job description which contain a PyTorch

In [136]:
data['PyTorch'] = data['fullJobDescribtion'].apply(lambda x: 1 if 'pytorch' in x.lower() else 0)

#### We want to create a new feature TensorFlow based on Job Description to get total of job description which contain a TensorFlow

In [137]:
data['TensorFlow'] = data['fullJobDescribtion'].apply(lambda x: 1 if 'tensorflow' in x.lower() or 'tf' in x.lower() else 0)

#### We want to create a new feature Tableau based on Job Description to get total of job description which contain a Tableau

In [138]:
data['Tableau'] = data['fullJobDescribtion'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)

#### We want to create a new feature Keras based on Job Description to get total of job description which contain a Keras

In [139]:
data['Keras'] = data['fullJobDescribtion'].apply(lambda x: 1 if 'keras' in x.lower() else 0)

#### We want to create a new feature Keras based on Job Description to get total of job description which contain a Keras

In [140]:
data['Keras'] = data['fullJobDescribtion'].apply(lambda x: 1 if 'keras' in x.lower() else 0)

#### We want to create a new feature Deep_Learning based on Job Description to get total of job description which contain a Deep Learning

In [141]:
data['Deep_Learning'] = data['fullJobDescribtion'].apply(lambda x:1 if ('deep learning' in x.lower() or 'dl' in x.lower()) else 0)

#### We want to create a new feature Machine_Deep based on Job Description to get total of job description which contain Deep Learning and Machine Learning 

In [142]:
data['Machine_Deep'] =((data['Deep_Learning'] ==1 ) & (data['Machine_Learning'] ==1 ))
data['Machine_Deep'] = data['Machine_Deep'].map({True:1,False:0})

#### We're gonna just make the CSV file after data cleaning

In [143]:
data.to_csv('data_after_cleaning.csv',index=False)